In [1]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
import random

from IPython.display import clear_output, HTML
import imageio

In [2]:
# GPU usage if not used by default (comment out if no GPU in use)
gpu = tf.config.list_physical_devices("GPU")
tf.config.set_visible_devices(gpu[0],"GPU")
print(f"Using GPU: {(gpu[0])}")

Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
env = gym.make("CartPole-v1",render_mode="rgb_array")
env.action_space.sample()

1

In [4]:
#play(env)
#display(show_video())

In [5]:
learning_rate=0.005
gama=0.90 # Discount factor
epsilon=1.0 # Exploration rate
epsilon_decay=0.99
epsilon_min=0.1
episodes=500

# Deep Q network
model=tf.keras.Sequential([
    tf.keras.layers.Input((env.observation_space.shape[0],)),
    tf.keras.layers.Dense(24,activation="relu"),
    tf.keras.layers.Dense(24,activation="relu"),
    tf.keras.layers.Dense(env.action_space.n,activation="linear")
])

loss=tf.keras.losses.MeanSquaredError()
optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate)
model.compile(loss=loss,optimizer=optimizer)

In [6]:
def get_action(state,model,epsilon):
    if np.random.rand() < epsilon:
        return env.action_space.sample()
    q_values=model.predict(state[np.newaxis],verbose=0)
    return np.argmax(q_values)


def play(env,model,epsilon):
    video_env = gym.wrappers.RecordVideo(env,video_folder="test",episode_trigger=lambda ep:True)
    
    state,_=video_env.reset()
    while True:
         # action=env.action_space.sample() # Random state
        action=get_action(state,model,epsilon)
        next_state,_,done,truncated,_=video_env.step(action)
        state=next_state
        if done or truncated:
            break
    
    video_env.close()
    return "test/rl-video-episode-0.mp4"


def show_video():
    clear_output(wait=True)
    return HTML(f"""
    <video width="640" height="480" controls>
        <source src="test/rl-video-episode-0.mp4" type="video/mp4">
    </video>
    """)

In [7]:
play(env,model,epsilon)

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:297: UserWarning: WARN: env.is_vector_env to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.is_vector_env` for environment variables or `env.get_attr('is_vector_env')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3121: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/lo

Moviepy - Building video /kaggle/working/test/rl-video-episode-0.mp4.
Moviepy - Writing video /kaggle/working/test/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /kaggle/working/test/rl-video-episode-0.mp4


'test/rl-video-episode-0.mp4'

In [ ]:
replay_buffer=[]

for episode in range(episodes):
    state,_=env.reset()
    total_reward=0

    while True:
        action=get_action(state,model,epsilon)
        next_state,reward,done,truncated,_=env.step(action)

        memory=(state,action,reward,next_state,done or truncated)
        replay_buffer.append(memory)
        if len(replay_buffer)>5000:
            replay_buffer.pop(0)

        if len(replay_buffer)>128:
            batch=random.sample(replay_buffer,128)

            states,actions,rewards,next_states,dones=zip(*batch)
            states=np.array(states,dtype=np.float32)
            rewards=np.array(rewards,dtype=np.float32)
            next_states=np.array(next_states,dtype=np.float32)
            dones=np.array(dones,dtype=np.float32)

            # Formula for calculating the right answer
            next_q=model.predict(next_states,verbose=0)
            target_q=model.predict(states,verbose=0)
            next_max_q=tf.reduce_max(next_q,axis=1)
            updated_q_values=rewards + (1 - dones) * gama * next_max_q

            for i in range(128):
                target_q[i,actions[i]]=updated_q_values.numpy()[i]

            with tf.GradientTape() as tape:
                q_values=model(states,training=True)
                loss=tf.keras.losses.MeanSquaredError()(target_q,q_values)
            gradients=tape.gradient(loss,model.trainable_variables)
            optimizer.apply_gradients(zip(gradients,model.trainable_variables))
        
        state=next_state
        total_reward+=reward
        if done or truncated:
            break
    
    epsilon=max(epsilon_min,epsilon*epsilon_decay)
    print(f"Episode: {episode}, Reward: {total_reward}, replay bufer: {len(replay_buffer)} epsilon: {epsilon}")
    
    if episode%10==0:
        play(env,model,0.0)
        display(show_video())

Episode: 11, Reward: 34.0, replay bufer: 280 epsilon: 0.8863848717161291
Episode: 12, Reward: 34.0, replay bufer: 314 epsilon: 0.8775210229989678
